In [17]:
#import sys
#import subprocess

#subprocess.check_call([sys.executable, "-m", "pip", "install", "pandas", "numpy", "statsmodels", "matplotlib", "scipy", "openpyxl"])

In [18]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import scipy.stats as stats

# Data Preparation

1) Load the cleaning function we created last time
2) Use the function to prepare the SPX and NVDA data

In [ ]:
# you can use this cleaning function we created last time
def data_cleaning(benchm, asset):

    benchm["Date"] = pd.to_datetime(benchm["Date"])
    benchm = benchm.sort_values(by='Date', ascending=True) 

    asset["Date"] = pd.to_datetime(asset["Date"])
    asset = asset.sort_values(by='Date', ascending=True) 

    comb = benchm.merge(asset,on="Date", how="left")

    comb['return_benchmark'] = comb.iloc[:, 1] / comb.iloc[:, 1].shift(1)-1
    comb['return_asset'] = comb.iloc[:, 2] / comb.iloc[:, 2].shift(1)-1
    #comb['logreturn_benchmark'] = np.log(comb.iloc[:, 1] / comb.iloc[:, 1].shift(1))
    #comb['logreturn_asset'] = np.log(comb.iloc[:, 2] / comb.iloc[:, 2].shift(1))
    comb = comb.dropna()

    return comb

3) Load the Fama-French 5-factor (FF5) time-series data
4) Format the dates to datetime format
5) Merge the SPX/NVDA data and the FF5 data using the SPX/NVDA dataframe as the leading dataframe
6) Compute excess returns for NVDA by subtracting the risk-free rate (column "RF" in FF5) from simple NVDA returns

# Multivariate Regression

1) Regress NVDA excess returns on a constant and all 5 Fama-French factors ("Mkt-RF", "SMB", "HML", "RMW", "CMA")

# FED Probit model

1) Load the dataset containing the difference between the 3-month and 10-year Treasury yield (t10y3m)
2) Ensure the dates are in datetime format and create a new column that contains the first date of the month to use it as a month indicator
3) Group data by that month indicator taking the monthly mean of the yield spread

4) Load the dataset with monthly recession indicators and ensure the date column is in datetime format

5) Merge the two dataframes using on the month indicator the yield spread as the leading dataframe
6) Create a new column that gives the 12 months ahead recession indicator using the .shift() method
7) Drop NaN observations

8) Estimate a probit model with an intercept, the yield spread as an independent variable and 12 month ahead recession indicator as the dependent variable using data until December 2009

9) Based on this model, predict the probability of a recession for each month in the entire dataset and plot that probability over time 